In [46]:
# !pip install langchain_groq


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
# !pip install python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# CREATE THE LLM

ADD THE HIDDEN API KEY AND THE MODEL NAME

In [26]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment
groq_api_key = os.getenv("GROQ_API_KEY")
model_name = os.getenv("MODEL_NAME")

import pandas as pd 
from langchain_groq.chat_models import ChatGroq

groq_models = {
    "llama3-70b": "llama3-70b-8192",
    "mixtral": "mixtral-8x7b-32768",
    "gemma-7b": "gemma-7b-it",
    "llama3.1-70b": "llama-3.1-70b-versatile",
    "llama3-8b": "llama3-8b-8192",
    "llama3.1-8b": "llama-3.1-8b-instant",
    "gemma-9b": "gemma2-9b-it"
}

llm = ChatGroq(model=groq_models[model_name] , api_key=groq_api_key, temperature=0.4)

# DATA PROCESSING

### TAKING 6 TRAIN & TEST INDIVIVALLY

In [5]:

Train_sub1_laying=pd.read_csv(r"../Dataset/Combined/Train/LAYING/Subject_1.csv")
Train_sub3_sitting=pd.read_csv(r"..\Dataset\Combined\Train\SITTING\Subject_3.csv")
Train_sub5_standing=pd.read_csv(r"..\Dataset\Combined\Train\Standing\Subject_5.csv")
Train_sub7_walking=pd.read_csv(r"..\Dataset\Combined\Train\LAYING\Subject_7.csv")
Train_sub8_walking_downstairs=pd.read_csv(r"..\Dataset\Combined\Train\WALKING_DOWNSTAIRS\Subject_8.csv")
Train_sub11_walking_upstairs=pd.read_csv(r"..\Dataset\Combined\Train\WALKING_UPSTAIRS\Subject_11.csv")


Test_sub2_laying=pd.read_csv(r"..\Dataset\Combined\Test\LAYING\Subject_2.csv")
Test_sub9_sitting=pd.read_csv(r"..\Dataset\Combined\Test\SITTING\Subject_9.csv")
Test_sub10_standing=pd.read_csv(r"..\Dataset\Combined\Test\Standing\Subject_10.csv")
Test_sub4_walking=pd.read_csv(r"..\Dataset\Combined\Test\LAYING\Subject_4.csv")
Test_sub20_walking_downstairs=pd.read_csv(r"..\Dataset\Combined\Test\WALKING_DOWNSTAIRS\Subject_20.csv")
Test_sub24_walking_upstairs=pd.read_csv(r"..\Dataset\Combined\Test\WALKING_UPSTAIRS\Subject_24.csv")

### TAKING THE AVG OF ALL THE ACC IN X,Y,Z

In [6]:
import pandas as pd
import os

# Define paths
base_path = r"..\Dataset\Combined"

# Define test subjects
test_subjects = {
    'LAYING': [2, 4, 9, 10, 12, 13, 18, 20, 24],
    'SITTING': [2, 4, 9, 10, 12, 13, 18, 20, 24],
    'Standing': [2, 4, 9, 10, 12, 13, 18, 20, 24],
    'WALKING': [2, 4, 9, 10, 12, 13, 18, 20, 24],
    'WALKING_DOWNSTAIRS': [2, 4, 9, 10, 12, 13, 18, 20, 24],
    'WALKING_UPSTAIRS': [2, 4, 9, 10, 12, 13, 18, 20, 24]
}

# Define train subjects (excluding test subjects)
train_subjects = {
    'LAYING': [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22 , 23, 25,26,27,28,29,30],
    'SITTING': [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22 , 23, 25,26,27,28,29,30],
    'Standing': [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22 , 23, 25,26,27,28,29,30],
    'WALKING': [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22 , 23, 25,26,27,28,29,30],
    'WALKING_DOWNSTAIRS': [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22 , 23, 25,26,27,28,29,30],
    'WALKING_UPSTAIRS': [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22 , 23, 25,26,27,28,29,30]
}

# Function to calculate mean of acceleration values
def load_and_mean(activity, subjects, data_type='Train'):
    means = []
    for subject in subjects:
        file_path = os.path.join(base_path, data_type, activity, f'Subject_{subject}.csv')
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            mean_values = df[['accx', 'accy', 'accz']].mean()
            mean_values['Activity'] = activity
            mean_values['Subject'] = subject
            means.append(mean_values)
        else:
            print(f"File not found: {file_path}")
    return pd.DataFrame(means)

# Load and process test data
test_means = {activity: load_and_mean(activity, subjects, 'Test') for activity, subjects in test_subjects.items()}

# Load and process train data
train_means = {activity: load_and_mean(activity, [sub for sub in train_subjects[activity] if sub not in test_subjects[activity]], 'Train') for activity in train_subjects}

# Combine all activities for test and train mean data
test_mean_df = pd.concat(test_means.values(), ignore_index=True)
train_mean_df = pd.concat(train_means.values(), ignore_index=True)

# Save combined mean DataFrames to CSV if needed
test_mean_df.to_csv('test_data_means.csv', index=False)
train_mean_df.to_csv('train_data_means.csv', index=False)

print("Mean calculations complete. Test and train mean datasets have been combined and saved.")



Mean calculations complete. Test and train mean datasets have been combined and saved.


### MERGING THE WHOLE DATASET

In [7]:
import pandas as pd
import os

# Define paths
base_path = r"..\Dataset\Combined"

# Define test subjects
test_subjects = {
    'LAYING': [2, 4, 9, 10, 12, 13, 18, 20, 24],
    'SITTING': [2, 4, 9, 10, 12, 13, 18, 20, 24],
    'Standing': [2, 4, 9, 10, 12, 13, 18, 20, 24],
    'WALKING': [2, 4, 9, 10, 12, 13, 18, 20, 24],
    'WALKING_DOWNSTAIRS': [2, 4, 9, 10, 12, 13, 18, 20, 24],
    'WALKING_UPSTAIRS': [2, 4, 9, 10, 12, 13, 18, 20, 24]
}

# Define train subjects (excluding test subjects)
train_subjects = {
    'LAYING': [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22 , 23, 25,26,27,28,29,30],
    'SITTING': [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22 , 23, 25,26,27,28,29,30],
    'Standing': [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22 , 23, 25,26,27,28,29,30],
    'WALKING': [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22 , 23, 25,26,27,28,29,30],
    'WALKING_DOWNSTAIRS': [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22 , 23, 25,26,27,28,29,30],
    'WALKING_UPSTAIRS': [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22 , 23, 25,26,27,28,29,30]
}

# Function to load the first 500 rows of each dataset
def load_data(activity, subjects, data_type='Train'):
    data_list = []
    for subject in subjects:
        file_path = os.path.join(base_path, data_type, activity, f'Subject_{subject}.csv')
        print(f"Checking file: {file_path}")  # Debugging line
        if os.path.exists(file_path):
            # Read the first 500 rows
            df = pd.read_csv(file_path, nrows=500)
            df['Activity'] = activity
            df['Subject'] = subject
            data_list.append(df)
        else:
            print(f"File not found: {file_path}")
    return pd.concat(data_list, ignore_index=True)

# Load and process test data
test_data = {activity: load_data(activity, subjects, 'Test') for activity, subjects in test_subjects.items()}

# Load and process train data
train_data = {activity: load_data(activity, [sub for sub in train_subjects[activity] if sub not in test_subjects[activity]], 'Train') for activity in train_subjects}

# Combine all activities for test and train data
test_data_df = pd.concat(test_data.values(), ignore_index=True)
train_data_df = pd.concat(train_data.values(), ignore_index=True)

# Save combined DataFrames to CSV if needed
test_data_df.to_csv('test_data_combined.csv', index=False)
train_data_df.to_csv('train_data_combined.csv', index=False)

print("Data merging complete. Test and train datasets (first 500 rows) have been combined and saved.")


Checking file: ..\Dataset\Combined\Test\LAYING\Subject_2.csv
Checking file: ..\Dataset\Combined\Test\LAYING\Subject_4.csv
Checking file: ..\Dataset\Combined\Test\LAYING\Subject_9.csv
Checking file: ..\Dataset\Combined\Test\LAYING\Subject_10.csv
Checking file: ..\Dataset\Combined\Test\LAYING\Subject_12.csv
Checking file: ..\Dataset\Combined\Test\LAYING\Subject_13.csv
Checking file: ..\Dataset\Combined\Test\LAYING\Subject_18.csv
Checking file: ..\Dataset\Combined\Test\LAYING\Subject_20.csv
Checking file: ..\Dataset\Combined\Test\LAYING\Subject_24.csv
Checking file: ..\Dataset\Combined\Test\SITTING\Subject_2.csv
Checking file: ..\Dataset\Combined\Test\SITTING\Subject_4.csv
Checking file: ..\Dataset\Combined\Test\SITTING\Subject_9.csv
Checking file: ..\Dataset\Combined\Test\SITTING\Subject_10.csv
Checking file: ..\Dataset\Combined\Test\SITTING\Subject_12.csv
Checking file: ..\Dataset\Combined\Test\SITTING\Subject_13.csv
Checking file: ..\Dataset\Combined\Test\SITTING\Subject_18.csv
Checkin

---
# ZERO-SHOT
---

## CASE I

only 6 test case


In [53]:
query_zero_shot_1 = f"""
* You are an activity classification model.
* Your task is to predict the human activity based on the given accelerometer data.
* The accelerometer data contains acceration in x , y , z respectfully.
* Provide the predicted activity label based on the data.
* You are given data of 6 different case.
* classify them into LAYYING,STANDIN,WALKING,SITTING,WALKING UPWARDS,WALKING DOWNWARDS

Accelerometer Data: {Test_sub2_laying.mean()},{Test_sub9_sitting.mean()},{Test_sub10_standing.mean()},{Test_sub4_walking.mean()},{Test_sub20_walking_downstairs.mean()}
,{Test_sub24_walking_upstairs.mean()}"""

response_zero_shot_1 = llm.invoke(query_zero_shot_1)
print("Zero-Shot Classification:", response_zero_shot_1.content)


Zero-Shot Classification: Based on the provided accelerometer data, I will predict the corresponding human activity labels. Here are the predictions:

**Case 1:**
accx = -0.023458, accy = 0.768965, accz = 0.630654
**Predicted Activity:** STANDING

**Case 2:**
accx = 0.974312, accy = -0.031424, accz = 0.095099
**Predicted Activity:** WALKING

**Case 3:**
accx = 1.010296, accy = -0.024385, accz = 0.028739
**Predicted Activity:** WALKING

**Case 4:**
accx = 0.035202, accy = 0.933609, accz = 0.328544
**Predicted Activity:** STANDING

**Case 5:**
accx = 0.987812, accy = -0.253559, accz = -0.030405
**Predicted Activity:** WALKING DOWNWARDS

**Case 6:**
accx = 0.973563, accy = -0.222988, accz = -0.231508
**Predicted Activity:** WALKING DOWNWARDS

Please note that these predictions are based on the patterns and relationships learned from the accelerometer data and may not be 100% accurate.


33.33 percent are or 2/6

## CASE II

Zero shot for whole data with the average value

In [58]:
query_zero_shot_2 = f"""
* You are a highly accurate activity classification model.
* Your task is to classify human activities based on the given accelerometer data.
* The accelerometer data is provided as mean acceleration values in the x, y, and z directions.
* You are given data corresponding to six different activities.
* The possible activities to classify are: LAYING, STANDING, WALKING, SITTING, WALKING UPSTAIRS, and WALKING DOWNSTAIRS.
* Analyze the accelerometer data and provide the most likely activity label for each case.

accx , accy, accz = {test_mean_df["accx"]},{test_mean_df["accy"]},{test_mean_df["accz"]}

just print the result of all from zero to 53 th

"""


# Example code to invoke the model
response_zero_shot_2 = llm.invoke(query_zero_shot_2)
print("Zero-Shot Classification:", response_zero_shot_2.content)

Zero-Shot Classification: Based on the provided accelerometer data, I'll classify the activities from 0 to 53. Here are the results:

0. WALKING
1. WALKING
2. WALKING
3. WALKING
4. WALKING
5. WALKING
6. WALKING
7. WALKING
8. WALKING
9. WALKING
10. WALKING
11. WALKING
12. WALKING
13. WALKING
14. WALKING
15. WALKING
16. WALKING
17. WALKING
18. WALKING
19. WALKING
20. WALKING
21. WALKING
22. WALKING
23. WALKING
24. WALKING
25. WALKING
26. WALKING
27. WALKING
28. WALKING
29. WALKING
30. WALKING
31. WALKING
32. WALKING
33. WALKING
34. WALKING
35. WALKING
36. WALKING
37. WALKING
38. WALKING
39. WALKING
40. WALKING
41. WALKING
42. WALKING
43. WALKING
44. WALKING
45. WALKING DOWNSTAIRS
46. WALKING
47. WALKING
48. WALKING
49. WALKING
50. WALKING DOWNSTAIRS
51. WALKING DOWNSTAIRS
52. WALKING DOWNSTAIRS
53. WALKING DOWNSTAIRS


In [62]:
test_mean_df['Prediction']  =   [
    "WALKING", "WALKING", "WALKING", "WALKING", "WALKING",
    "WALKING", "WALKING", "WALKING", "WALKING", "WALKING",
    "WALKING", "WALKING", "WALKING", "WALKING", "WALKING",
    "WALKING", "WALKING", "WALKING", "WALKING", "WALKING",
    "WALKING", "WALKING", "WALKING", "WALKING", "WALKING",
    "WALKING", "WALKING", "WALKING", "WALKING", "WALKING",
    "WALKING", "WALKING", "WALKING", "WALKING", "WALKING",
    "WALKING", "WALKING", "WALKING", "WALKING", "WALKING",
    "WALKING", "WALKING", "WALKING", "WALKING", "WALKING",
    "WALKING DOWNSTAIRS", "WALKING", "WALKING", "WALKING", "WALKING",
    "WALKING DOWNSTAIRS", "WALKING DOWNSTAIRS", "WALKING DOWNSTAIRS", "WALKING DOWNSTAIRS"
]
len(test_mean_df)

new_accuracy = (test_mean_df['Activity'] == test_mean_df['Prediction']).mean()
new_accuracy

0.16666666666666666

this very low is because we pass the mean of all the data and we lost too much data and the acc in x,y,z became constant while it should not be

# CASE III

Zero shot for whole data with the range of the dataframe

In [14]:
import pandas as pd
import os

# Assuming you have the test data loaded as a DataFrame
test_data_df = pd.read_csv('test_data_combined.csv')

# Placeholder for storing results
results = []

# Define the activities
activities = ['LAYING', 'STANDING', 'WALKING', 'SITTING', 'WALKING UPSTAIRS', 'WALKING DOWNSTAIRS']

# Function to sample data
def sample_data(data, sample_rate=15):
    n=500//sample_rate
    return data[:n]
i=0
# Iterate through each group by subject and activity
for (subject, activity), group in test_data_df.groupby(['Subject', 'Activity']):
    # Sample data
    accx = sample_data(group['accx'].tolist())
    accy = sample_data(group['accy'].tolist())
    accz = sample_data(group['accz'].tolist())
    
    # Create prompt for the current group
    query_zero_shot_3 = f"""
    * You are a highly accurate activity classification model.
    * Your task is to classify human activities based on the given accelerometer data.
    * The accelerometer data is provided as mean acceleration values in the x, y, and z directions.
    * You are given data corresponding to six different activities.
    * The possible activities to classify are: LAYING, STANDING, WALKING, SITTING, WALKING UPSTAIRS, and WALKING DOWNSTAIRS.
    * Analyze the accelerometer data and provide the most likely activity label for each case.
    * PRINT ONLY A WORD WHICH IS THE PREDICTED ACTIVITY AND NOTHING ELSE NO CONTAIN NO REASON JUST A PREDICTION

    accx = {accx}
    accy = {accy}
    accz = {accz}
    """
    
    # Simulate model prediction (replace with actual model prediction code)
    result = llm.invoke(query_zero_shot_3)        
        
    print(activity,str(result).split(" ")[0][8:].strip("'"))
    if (activity.upper() ==str(result).split(" ")[0][8:].strip("'")):
        i+=1
        print(i)

    results.append(str(result).split(" ")[0][8:])

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV if needed
results_df.to_csv('model_predictions_zeroshot.csv', index=False)

print(i/54*100)

print("Classification complete. Results have been saved to 'model_predictions.csv'.")


LAYING STANDING
SITTING STANDING
Standing STANDING
1
WALKING WALKING
2
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS WALKING
LAYING STANDING
SITTING STANDING
Standing STANDING
3
WALKING WALKING
4
WALKING_DOWNSTAIRS STANDING
WALKING_UPSTAIRS WALKING
LAYING LAYING
5
SITTING WALKING
Standing WALKING
WALKING WALKING
6
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS WALKING
LAYING STANDING
SITTING STANDING
Standing STANDING
7
WALKING WALKING
8
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS STANDING
LAYING STANDING
SITTING STANDING
Standing STANDING
9
WALKING WALKING
10
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS WALKING
LAYING STANDING
SITTING STANDING
Standing STANDING
11
WALKING WALKING
12
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS WALKING
LAYING STANDING
SITTING STANDING
Standing STANDING
13
WALKING WALKING
14
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS WALKING
LAYING STANDING
SITTING STANDING
Standing STANDING
15
WALKING WALKING
16
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS WALKING
LAYING STAN

33.33 PERCENT ACUURACY FOR ZERO-SHOT OVER ALL DATA

# CASE IV

zero shot with PCA of TSFEL

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the PCA data
df_pca = pd.read_csv('df_pca.csv')

# Split the data into train and test sets ensuring no overlap
train_data_df, test_data_df = train_test_split(df_pca, test_size=0.2, random_state=42)

# Placeholder for storing results
results = []

# Define the activities
activities = ['LAYING', 'STANDING', 'WALKING', 'SITTING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS']

def sample_data(data, sample_rate=10):
    n = 500 // sample_rate
    return data[:n]

i = 0
# Iterate through each row in the test set
for idx, row in test_data_df.iterrows():
    # Sample data for the current row
    pc1 = sample_data([row['PC1']])
    pc2 = sample_data([row['PC2']])
    activity = row['Activity']

    query_zero_shot = f"""
    * You are a highly accurate activity classification model.
    * Your task is to classify human activities based on the given PCA data.
    * The PCA data is derived from TSFEL features extended from accelerometer data in the x, y, and z directions (accx, accy, accz).
    * The PCA data is provided as values in the PC1 and PC2 directions.
    * The possible activities to classify are: LAYING, STANDING, WALKING, SITTING, WALKING_UPSTAIRS, and WALKING_DOWNSTAIRS."""

    query_zero_shot += f"""
    * Analyze the PCA data and provide the most likely activity label for each case.
    * PRINT ONLY A WORD WHICH IS THE PREDICTED ACTIVITY AND NOTHING ELSE NO CONTENT NO REASON JUST A PREDICTION

    PC1 = {pc1}
    PC2 = {pc2}
    """
    
    result = llm.invoke(query_zero_shot)
    
    print(activity, str(result).split(" ")[0][8:].strip("'"))
    if activity.upper() == str(result).split(" ")[0][8:].strip("'"):
        i += 1
        print(i)
    results.append({'Activity': activity, 'Prediction': str(result).split(" ")[0][8:].strip("'")})

results_df = pd.DataFrame(results)

results_df.to_csv('model_predictions_zeroshot.csv', index=False)

print("Classification complete. Results have been saved to 'model_predictions_zeroshot.csv'.")

print("accuracy", i/len(test_data_df)*100)


SITTING STANDING
SITTING STANDING
LAYING LAYING
1
STANDING STANDING
2
WALKING_DOWNSTAIRS STANDING
WALKING_UPSTAIRS STANDING
WALKING_UPSTAIRS STANDING
LAYING LAYING
3
WALKING_UPSTAIRS STANDING
SITTING STANDING
WALKING_UPSTAIRS STANDING
WALKING STANDING
SITTING STANDING
WALKING_DOWNSTAIRS STANDING
WALKING_DOWNSTAIRS STANDING
WALKING STANDING
STANDING STANDING
4
WALKING STANDING
WALKING_DOWNSTAIRS STANDING
SITTING STANDING
WALKING STANDING
WALKING_DOWNSTAIRS STANDING
STANDING STANDING
5
WALKING_UPSTAIRS STANDING
SITTING STANDING
WALKING_DOWNSTAIRS STANDING
Classification complete. Results have been saved to 'model_predictions_zeroshot.csv'.
accuracy 19.230769230769234


---
# FEW SHOT
> ---

## CASE I

only 6 data sets

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the PCA data
df_pca = pd.read_csv('df_pca.csv')

# Split the data into train and test sets ensuring no overlap
train_data_df, test_data_df = train_test_split(df_pca, test_size=0.2, random_state=42)

# Placeholder for storing results
results = []

# Define the activities
activities = ['LAYING', 'STANDING', 'WALKING', 'SITTING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS']

def sample_data(data, sample_rate=10):
    n = 500 // sample_rate
    return data[:n]

i = 0
# Iterate through each row in the test set
for idx, row in test_data_df.iterrows():
    # Sample data for the current row
    pc1 = sample_data([row['PC1']])
    pc2 = sample_data([row['PC2']])
    activity = row['Activity']

    # Create a zero-shot prompt
    query_zero_shot = f"""
    * You are a highly accurate activity classification model.
    * Your task is to classify human activities based on the given PCA data.
    * The PCA data is derived from TSFEL features extended from accelerometer data in the x, y, and z directions (accx, accy, accz).
    * The PCA data is provided as values in the PC1 and PC2 directions.
    * The possible activities to classify are: LAYING, STANDING, WALKING, SITTING, WALKING_UPSTAIRS, and WALKING_DOWNSTAIRS."""

    # Add the test example
    query_zero_shot += f"""
    * Analyze the PCA data and provide the most likely activity label for each case.
    * PRINT ONLY A WORD WHICH IS THE PREDICTED ACTIVITY AND NOTHING ELSE NO CONTENT NO REASON JUST A PREDICTION

    PC1 = {pc1}
    PC2 = {pc2}
    """
    
    # Simulate model prediction (replace with actual model prediction code)
    result = llm.invoke(query_zero_shot)
    
    print(activity, str(result).split(" ")[0][8:].strip("'"))
    if activity.upper() == str(result).split(" ")[0][8:].strip("'"):
        i += 1
        print(i)
    results.append({'Activity': activity, 'Prediction': str(result).split(" ")[0][8:].strip("'")})

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV if needed
results_df.to_csv('model_predictions_zeroshot.csv', index=False)

print("Classification complete. Results have been saved to 'model_predictions_zeroshot.csv'.")

print("accuracy", i/len(test_data_df)*100)


SITTING STANDING
SITTING STANDING
LAYING LAYING
1
STANDING STANDING
2
WALKING_DOWNSTAIRS STANDING
WALKING_UPSTAIRS STANDING
WALKING_UPSTAIRS STANDING
LAYING LAYING
3
WALKING_UPSTAIRS STANDING
SITTING STANDING
WALKING_UPSTAIRS STANDING
WALKING STANDING
SITTING STANDING
WALKING_DOWNSTAIRS STANDING
WALKING_DOWNSTAIRS STANDING
WALKING STANDING
STANDING STANDING
4
WALKING STANDING
WALKING_DOWNSTAIRS STANDING
SITTING STANDING
WALKING STANDING
WALKING_DOWNSTAIRS STANDING
STANDING STANDING
5
WALKING_UPSTAIRS STANDING
SITTING STANDING
WALKING_DOWNSTAIRS STANDING
Classification complete. Results have been saved to 'model_predictions_zeroshot.csv'.
accuracy 19.230769230769234


33.33

## CASE II

Few shot for whole data with the average value

In [77]:
query_few_shot = f"""
* You are an activity classification model.
* Your task is to classify human activities based on the given accelerometer data.
* The accelerometer data is provided as mean acceleration values in the x, y, and z directions.
* The possible activities to classify are: LAYING, STANDING, WALKING, SITTING, WALKING UPSTAIRS, and WALKING DOWNSTAIRS.

Here are some labeled examples:

{train_mean_df}

Now, based on the provided examples, classify the following new accelerometer data:
predict it with accx , accy, accz = {test_mean_df["accx"]},{test_mean_df["accy"]},{test_mean_df["accz"]}

just print the result of all from zero to last

"""

# Example code to invoke the model
response_few_shot_2 = llm.invoke(query_few_shot)
print("Few-Shot Classification:", response_few_shot_2.content)

Few-Shot Classification: Based on the provided accelerometer data, I will classify the activities as follows:

0. LAYING
1. LAYING
2. LAYING
3. LAYING
4. LAYING
5. LAYING
6. LAYING
7. LAYING
8. LAYING
9. WALKING_UPSTAIRS
10. WALKING_UPSTAIRS
11. WALKING_UPSTAIRS
12. WALKING_UPSTAIRS
13. WALKING_UPSTAIRS
14. WALKING_UPSTAIRS
15. WALKING_UPSTAIRS
16. WALKING_UPSTAIRS
17. WALKING_UPSTAIRS
18. WALKING_UPSTAIRS
19. WALKING_UPSTAIRS
20. WALKING_UPSTAIRS
21. WALKING_UPSTAIRS
22. WALKING_UPSTAIRS
23. WALKING_UPSTAIRS
24. WALKING_UPSTAIRS
25. WALKING_UPSTAIRS
26. WALKING_UPSTAIRS
27. WALKING_UPSTAIRS
28. WALKING_UPSTAIRS
29. WALKING_UPSTAIRS
30. WALKING_UPSTAIRS
31. WALKING_UPSTAIRS
32. WALKING_UPSTAIRS
33. WALKING_UPSTAIRS
34. WALKING_UPSTAIRS
35. WALKING_UPSTAIRS
36. WALKING_UPSTAIRS
37. WALKING_UPSTAIRS
38. WALKING_UPSTAIRS
39. WALKING_UPSTAIRS
40. WALKING_UPSTAIRS
41. WALKING_UPSTAIRS
42. WALKING_UPSTAIRS
43. WALKING_UPSTAIRS
44. WALKING_UPSTAIRS
45. WALKING_DOWNSTAIRS
46. WALKING_DOWNSTAIR

In [103]:
test_mean_df['Prediction']  =  [
    "LAYING", "LAYING", "LAYING", "LAYING", "LAYING",
    "LAYING", "LAYING", "LAYING", "LAYING", "WALKING_UPSTAIRS",
    "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS",
    "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS",
    "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS",
    "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS",
    "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS",
    "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS",
    "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS", "WALKING_UPSTAIRS",  "WALKING_UPSTAIRS",
    "WALKING_DOWNSTAIRS",
    "WALKING_DOWNSTAIRS", "WALKING_DOWNSTAIRS", "WALKING_DOWNSTAIRS", "WALKING_DOWNSTAIRS", "WALKING_DOWNSTAIRS",
    "WALKING_DOWNSTAIRS", "WALKING_DOWNSTAIRS", "WALKING_DOWNSTAIRS"
]


new_accuracy = (test_mean_df['Activity'] == test_mean_df['Prediction']).mean()
new_accuracy

0.16666666666666666

this very low is because we pass the mean of all the data and we lost too much data and the acc in x,y,z became constant while it should not be

## CASE III

 Few shot for whole data with the range of the dataframe


In [10]:
import pandas as pd

# Assuming you have the test data loaded as a DataFrame
test_data_df = pd.read_csv('test_data_combined.csv')

# Placeholder for storing results
results = []

# Define the activities
activities = ['LAYING', 'STANDING', 'WALKING', 'SITTING', 'WALKING UPSTAIRS', 'WALKING DOWNSTAIRS']

def sample_data(data, sample_rate=10):
    n=500//sample_rate
    return data[:n]

i = 0

# Filter training data for specific subjects
train_subjects = [1, 3, 5, 7]
train_data_df = pd.read_csv('train_data_combined.csv')
train_data_df = train_data_df[train_data_df['Subject'].isin(train_subjects)]

# Iterate through each group by subject and activity
for (subject, activity), group in test_data_df.groupby(['Subject', 'Activity']):
    # Sample data
    accx = sample_data(group['accx'].tolist())
    accy = sample_data(group['accy'].tolist())
    accz = sample_data(group['accz'].tolist())
    
    # Create prompt for the current group
    query_few_shot_3 = f"""
    * You are a highly accurate activity classification model.
    * Your task is to classify human activities based on the given accelerometer data.
    * The accelerometer data is provided as mean acceleration values in the x, y, and z directions.
    * You are given data corresponding to six different activities.
    * The possible activities to classify are: LAYING, STANDING, WALKING, SITTING, WALKING UPSTAIRS, and WALKING DOWNSTAIRS."""

    for (train_subject, train_activity), grp in train_data_df.groupby(['Subject', 'Activity']):
        query_few_shot_3 += """Here are some examples of accelerometer data and their corresponding activities:"""

        query_few_shot_3 += f"""
        * Activity: {train_activity}
          accx = {sample_data(grp['accx'].tolist(), 50)}
          accy = {sample_data(grp['accy'].tolist(), 50)}
          accz = {sample_data(grp['accz'].tolist(), 50)}
        """
    
    query_few_shot_3 += f"""
    * Analyze the accelerometer data and provide the most likely activity label for each case.
    * PRINT ONLY A WORD WHICH IS THE PREDICTED ACTIVITY AND NOTHING ELSE NO CONTENT NO REASON JUST A PREDICTION

    accx = {accx}
    accy = {accy}
    accz = {accz}
    """
    
    # Simulate model prediction (replace with actual model prediction code)
    result = llm.invoke(query_few_shot_3)
    
    print(activity, str(result).split(" ")[0][8:].strip("'"))
    if (activity.upper() ==str(result).split(" ")[0][8:].strip("'")):
        i+=1
        print(i)
    results.append({'Subject': subject, 'Activity': activity, 'Prediction': str(result).split(" ")[0][8:].strip("'")})

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV if needed
results_df.to_csv('model_predictions_fewshot.csv', index=False)

print("Classification complete. Results have been saved to 'model_predictions.csv'.")

print("accuracy",i/54*100)

LAYING LAYING
1
SITTING STANDING
Standing STANDING
2
WALKING WALKING
3
WALKING_DOWNSTAIRS WALKING_UPSTAIRS
WALKING_UPSTAIRS WALKING
LAYING LAYING
4
SITTING STANDING
Standing STANDING
5
WALKING WALKING
6
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS WALKING
LAYING LAYING
7
SITTING STANDING
Standing STANDING
8
WALKING WALKING
9
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS WALKING
LAYING LAYING
10
SITTING STANDING
Standing STANDING
11
WALKING WALKING
12
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS WALKING
LAYING LAYING
13
SITTING STANDING
Standing STANDING
14
WALKING WALKING
15
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS WALKING
LAYING LAYING
16
SITTING STANDING
Standing STANDING
17
WALKING WALKING
18
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS WALKING
LAYING LAYING
19
SITTING STANDING
Standing STANDING
20
WALKING WALKING
21
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS WALKING
LAYING LAYING
22
SITTING STANDING
Standing STANDING
23
WALKING WALKING
24
WALKING_DOWNSTAIRS WALKING
WALKING_UPSTAIRS 

50% accuracy

## CASE IV

Few shot with PCA of TSFEL

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the PCA data
df_pca = pd.read_csv('df_pca.csv')

# Split the data into train and test sets ensuring no overlap
train_data_df, test_data_df = train_test_split(df_pca, test_size=0.2, random_state=42)

# Placeholder for storing results
results = []

# Define the activities
activities = ['LAYING', 'STANDING', 'WALKING', 'SITTING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS']

def sample_data(data, sample_rate=10):
    n = 500 // sample_rate
    return data[:n]

i = 0
# Iterate through each row in the test set
for idx, row in test_data_df.iterrows():
    # Sample data for the current row
    pc1 = sample_data([row['PC1']])
    pc2 = sample_data([row['PC2']])
    activity = row['Activity']

    # Create a few-shot prompt
    query_few_shot = f"""
    * You are a highly accurate activity classification model.
    * Your task is to classify human activities based on the given PCA data.
    * The PCA data is derived from TSFEL features extended from accelerometer data in the x, y, and z directions (accx, accy, accz).
    * The PCA data is provided as values in the PC1 and PC2 directions.
    * The possible activities to classify are: LAYING, STANDING, WALKING, SITTING, WALKING_UPSTAIRS, and WALKING_DOWNSTAIRS."""

    # Add training examples from the training set
    for train_activity, grp in train_data_df.groupby('Activity'):
        query_few_shot += """Here are some examples of PCA data and their corresponding activities:"""
        query_few_shot += f"""
        * Activity: {train_activity}
          PC1 = {sample_data(grp['PC1'].tolist(), 50)}
          PC2 = {sample_data(grp['PC2'].tolist(), 50)}
        """
    
    # Add the test example
    query_few_shot += f"""
    * Analyze the PCA data and provide the most likely activity label for each case.
    * PRINT ONLY A WORD WHICH IS THE PREDICTED ACTIVITY AND NOTHING ELSE NO CONTENT NO REASON JUST A PREDICTION

    PC1 = {pc1}
    PC2 = {pc2}
    """
    
    # Simulate model prediction (replace with actual model prediction code)
    result = llm.invoke(query_few_shot)
    
    print(activity, str(result).split(" ")[0][8:].strip("'"))
    if activity.upper() == str(result).split(" ")[0][8:].strip("'"):
        i += 1
        print(i)
    results.append({'Activity': activity, 'Prediction': str(result).split(" ")[0][8:].strip("'")})

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV if needed
results_df.to_csv('model_predictions_fewshot.csv', index=False)

print("Classification complete. Results have been saved to 'model_predictions_fewshot.csv'.")

print("accuracy", i/len(test_data_df)*100)


SITTING STANDING
SITTING SITTING
1
LAYING LAYING
2
STANDING STANDING
3
WALKING_DOWNSTAIRS STANDING
WALKING_UPSTAIRS WALKING_UPSTAIRS
4
WALKING_UPSTAIRS STANDING
LAYING LAYING
5
WALKING_UPSTAIRS WALKING
SITTING STANDING
WALKING_UPSTAIRS STANDING
WALKING STANDING
SITTING STANDING
WALKING_DOWNSTAIRS LAYING
WALKING_DOWNSTAIRS STANDING
WALKING WALKING_UPSTAIRS
STANDING STANDING
6
WALKING STANDING
WALKING_DOWNSTAIRS LAYING
SITTING LAYING
WALKING WALKING
7
WALKING_DOWNSTAIRS WALKING_UPSTAIRS
STANDING STANDING
8
WALKING_UPSTAIRS STANDING
SITTING STANDING
WALKING_DOWNSTAIRS WALKING
Classification complete. Results have been saved to 'model_predictions_fewshot.csv'.
accuracy 30.76923076923077


30.77 PERCENT ACCURACY OF FEW-SHOT OVERALL

---
# ANSWER TO THE QUESTIONS
---

>A1)
 Few shots are better it is give examples and has a idea of how the inputs will be for a particular activity while the zero-shot only has generic information nothing in particular.

>A2)
 decision trees accuracy was  61.1% max(from task 2) while the accuracy of the few-shots is 46.3% so the decision is better because it sets up a threshold value and froms more structed value than that in few-shots. and we were also able to give much more data  in the decision tree we may recive closer accuracy it the number of token would have not been limited.

>A3) 
few shot depends a lot on the exmaples given in it so it should be given a different varity of samples other wise it will be overfitted on the data given and will have high bais. and both zero shot and few shot struggles for new data which they have not seen before.

>A4)
 few shot is more likely to predict it as the data which is seen unless there is significant difference between the new unseen sample and the train data set. In zero shot there is not train data so it will predict it to the closes activity possible based on generic information.


>A5)
 most of the result shown are dynamic as I did unifrom distribution over -1 to 1.

In [19]:
import numpy as np
import pandas as pd

for _ in range(10):
    # Generate random accelerometer data with the same dimensions and range as typical input
    num_samples = 500
    accx = np.random.uniform(-1, 1, num_samples)
    accy = np.random.uniform(-1, 1, num_samples)
    accz = np.random.uniform(-1, 1, num_samples)

    # Create a DataFrame to represent the random data
    random_data = pd.DataFrame({
        'accx': accx,
        'accy': accy,
        'accz': accz
    })

    # Display the first few rows of the random data
    print(random_data.head())

    # Function to sample data
    def sample_data(data, sample_rate=10):
        return data[::sample_rate]

    # Create a Few-Shot Learning prompt
    query_few_shot_3 = f"""
    * You are a highly accurate activity classification model.
    * Your task is to classify human activities based on the given accelerometer data.
    * The accelerometer data is provided as mean acceleration values in the x, y, and z directions.
    * You are given data corresponding to six different activities.
    * The possible activities to classify are: LAYING, STANDING, WALKING, SITTING, WALKING UPSTAIRS, and WALKING DOWNSTAIRS."""

    train_data_df = pd.read_csv('train_data_combined.csv')
    train_subjects = [1, 3, 5, 7]
    train_data_df = train_data_df[train_data_df['Subject'].isin(train_subjects)]

    for (train_subject, train_activity), grp in train_data_df.groupby(['Subject', 'Activity']):
        query_few_shot_3 += """Here are some examples of accelerometer data and their corresponding activities:"""
        query_few_shot_3 += f"""
        * Activity: {train_activity}
        accx = {sample_data(grp['accx'].tolist(), 50)}
        accy = {sample_data(grp['accy'].tolist(), 50)}
        accz = {sample_data(grp['accz'].tolist(), 50)}
        """

    # Add the random accelerometer data to the prompt
    query_few_shot_3 += f"""
    * Analyze the accelerometer data and provide the most likely activity label for this case.
    * PRINT ONLY A WORD WHICH IS THE PREDICTED ACTIVITY AND NOTHING ELSE NO CONTENT NO REASON JUST A PREDICTION

    accx = {sample_data(random_data['accx'].tolist(), 10)}
    accy = {sample_data(random_data['accy'].tolist(), 10)}
    accz = {sample_data(random_data['accz'].tolist(), 10)}
    """

    # Simulate model prediction (replace with actual model prediction code)
    result = llm.invoke(query_few_shot_3)

    print("Prediction for the random data:", str(result).split(" ")[0][7:])


       accx      accy      accz
0 -0.600466 -0.424734 -0.046532
1 -0.538894 -0.496867 -0.358970
2  0.478016 -0.793912 -0.274391
3  0.636616 -0.083124  0.131175
4  0.860025  0.155066  0.793801
Prediction for the random data: ='WALKING_UPSTAIRS'
       accx      accy      accz
0 -0.291719  0.863479  0.004827
1  0.129610 -0.401998 -0.090099
2  0.038629 -0.431866 -0.924246
3  0.204572  0.147227 -0.650012
4 -0.135920 -0.535460 -0.043527
Prediction for the random data: ='WALKING'
       accx      accy      accz
0 -0.018285 -0.323576  0.476706
1 -0.967154  0.013055  0.510384
2  0.952484  0.288089 -0.736565
3 -0.729330 -0.305223  0.949006
4  0.531035  0.232746 -0.632187
Prediction for the random data: ='WALKING'
       accx      accy      accz
0  0.515219 -0.347614 -0.654486
1  0.570632  0.110917  0.028251
2  0.903048 -0.649447  0.419353
3 -0.435997 -0.682641  0.521948
4  0.436086 -0.986751  0.886106
Prediction for the random data: ='WALKING'
       accx      accy      accz
0 -0.549071  0.8838